In [65]:
import pandas as pd
import numpy as np

In [66]:
n_profiles = pd.read_csv('neighbourhood_profiles_2016.csv')

In [278]:
n_profiles.head()

,Topic,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Number,NaN,129,128,20,95,42,34,76,...,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,Population and dwellings,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526","27,695","15,873","25,797",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,Population and dwellings,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177","26,918","15,434","19,348",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,Population and dwellings,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,2.90%,2.80%,33.30%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


In [67]:
n_profiles = n_profiles.drop(['_id','Category','Data Source'],axis=1)

In [100]:
df = pd.melt(n_profiles, id_vars=['Topic','Characteristic'], value_vars=n_profiles.columns[2:])

In [101]:
df.columns = ['topic','characterstic','neighbourhood','value']

In [279]:
df.head()

,topic,characterstic,neighbourhood,value
0,Neighbourhood Information,Neighbourhood Number,City of Toronto,NaN
1,Neighbourhood Information,TSNS2020 Designation,City of Toronto,NaN
2,Population and dwellings,"Population, 2016",City of Toronto,"2,731,571"
3,Population and dwellings,"Population, 2011",City of Toronto,"2,615,060"
4,Population and dwellings,Population Change 2011-2016,City of Toronto,4.50%


### Population

In [103]:
pop = df[df['topic']=='Population and dwellings']

In [104]:
pop = pop[pop['characterstic']=='Population, 2016']

In [105]:
pop = pop[['neighbourhood','value']]

In [107]:
pop.columns = ['neighbourhood','population']

### Population Density

In [111]:
pop_dens = df[df['topic']=='Population and dwellings']
pop_dens = pop_dens[pop_dens['characterstic']=='Population density per square kilometre']
pop_dens = pop_dens[['neighbourhood','value']]
pop_dens.columns = ['neighbourhood','population_density']

### Land Area

In [119]:
area = df[df['topic']=='Population and dwellings']
area = area[area['characterstic']=='Land area in square kilometres']
area = area[['neighbourhood','value']]
area.columns = ['neighbourhood','area']

### Household Income

In [154]:
inc = df[df['topic']=='Income of households in 2015'] 
inc = inc[inc['characterstic']=='  Average after-tax income of households in 2015 ($)']
inc = inc[['neighbourhood','value']]
inc.columns = ['neighbourhood','median_income_household']

### Age

In [271]:
age_to_keep = ['Children (0-14 years)', 'Youth (15-24 years)',
       'Working Age (25-54 years)', 'Pre-retirement (55-64 years)',
       'Seniors (65+ years)', 'Older Seniors (85+ years)']

age = df[df['topic']=='Age characteristics'] 
age = age[age['characterstic'].isin(age_to_keep)]
age = age[['neighbourhood','characterstic','value']]
age.columns = ['neighbourhood','age_group','percentage']
age['percentage'] = age['percentage'].str.replace(',','')
age['percentage'] = pd.to_numeric(age['percentage'], errors='coerce')

In [272]:
age_total = age.groupby(['neighbourhood'])['percentage'].sum().to_frame('total').reset_index()

age = age.merge(age_total,on='neighbourhood')

In [273]:
age['percentage'] = ((age['percentage']/age['total'])*100).round(2)

In [274]:
age = age[['neighbourhood','age_group','percentage']]

In [275]:
age = pd.pivot(age,index='neighbourhood',columns='age_group',values='percentage').reset_index().rename_axis(None, axis=1)

age.columns = ['neighbourhood','age_0-14','age_85+','age_55-64','age_65+','age_25-54','age_15-24']

age = age[['neighbourhood','age_0-14','age_15-24','age_25-54','age_55-64','age_65+','age_85+']]

In [276]:
age

,neighbourhood,age_0-14,age_15-24,age_25-54,age_55-64,age_65+,age_85+
0,Agincourt North,12.78,12.33,37.62,14.08,20.12,3.08
1,Agincourt South-Malvern West,12.64,13.81,40.97,13.42,16.88,2.28
2,Alderwood,14.22,9.98,42.18,14.75,16.28,2.59
3,Annex,7.47,11.87,47.63,11.02,18.71,3.29
4,Banbury-Don Mills,12.30,9.31,36.88,12.13,23.79,5.59
...,...,...,...,...,...,...,...
136,Wychwood,12.22,8.67,42.17,10.48,20.69,5.78
137,Yonge-Eglinton,15.03,10.23,48.94,11.06,13.36,1.38
138,Yonge-St.Clair,9.30,7.07,45.83,11.84,22.34,3.61
139,York University Heights,14.46,16.98,43.92,10.60,12.62,1.43
